# Train model to detect boulders

In [12]:
# We use the darknet github
# https://github.com/AlexeyAB/darknet
# It also requires an installation of the gdal utilities, to be accessible from the command line (e.g.,
# need to be able to execute "gdal_retile.py on the command line")

# Make folder structure for current run
!mkdir temp
!mkdir temp/train
!mkdir temp/test
!mkdir temp/current_run 
!mkdir temp/empty


In [1]:
import os

In [13]:
# Cut training mosaics into small tiles with overlap
# Cutting around the individual boulders would also be possible (cut_image_around_point.py, see comments in file) 
# but this would mean many examples are present multiple times.

!python cut_image_to_tiles.py ./mosaics_for_train ./temp/train 64 tif --overlap=4

Using  15  cores. 
Tiling Mosaics:
1.tif
  0%|                                                     | 0/1 [00:00<?, ?it/s]Cutting with Overlap
0...10...20...30...40...50...60...70...80...90...100 - done.
100%|█████████████████████████████████████████████| 1/1 [00:40<00:00, 40.37s/it]


In [14]:
# Delete images with low standard deviation (get rid of the background)

!python delete_low_stddev_images.py ./temp/train .tif 4.0

Using  15  cores. 
Working on  5092  files.
100%|██████████████████████████████████████| 5092/5092 [00:07<00:00, 644.02it/s]


In [15]:
# Split files into test and train set

!python create_random_distribution.py ./temp/train ./temp/test 0.10 tif

Splitting all files in folder: ./temp/train
100%|███████████████████████████████████████| 460/460 [00:00<00:00, 8908.19it/s]


In [ ]:
# Upscaling would be here

In [16]:
# Relate available Tiles with manually picked boulder positions to create test and training datasets
# This is done separately for the test and train folders
# Also, this needs to be done separately for each class
# "Empty/Negative" image examples are special and come in the next step

##!!!! the yolo classes at the moment use hardcoded integers. So ONLY 1 CLASS AT THE MOMENT

!python relate_database_and_images.py \
--image_directory=./temp/train/ \
--wildcards=tif \
--database_directory=./vector_data/ \
--input_databases=v2pickedstone_utm.sqlite \
--input_classes=stone \
--out_directory=./temp/train/ \
--format=yolo

!python relate_database_and_images.py \
--image_directory=./temp/test/ \
--wildcards=tif \
--database_directory=./vector_data/ \
--input_databases=v2pickedstone_utm.sqlite \
--input_classes=stone \
--out_directory=./temp/test/ \
--format=yolo


['v2pickedstone_utm.sqlite']
['v2pickedstone_utm']
relate point to mosaic:
Working on:  v2pickedstone_utm.sqlite stone v2pickedstone_utm
Working on  7988 training examples
Comparing with  4146 images
Found  4646  matches
Make annotated csv files
/Users/peter/anaconda/anaconda3/envs/geo/lib/python3.6/site-packages/pandas/core/indexing.py:659: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[k] = np.nan
relate_database_and_images.py:204: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp[['x_center_norm

In [17]:
#Add negative image examples
#This is a bit hacky
#Negative example are just points in an SQLite database
#Again this is done for the test and train folder

!python relate_database_and_images.py \
--image_directory=./temp/train/ \
--wildcards=tif \
--database_directory=./vector_data/ \
--input_databases=v2empty_utm.sqlite\
--input_classes=empty \
--out_directory=./temp/empty/ \
--format=yolo \
--empty_examples=1

# Empty images need empty text files with the same name for YOLO
!for file in ./temp/empty/*.txt; do rm $file; touch $file; done
# Move these files to train folder
!mv ./temp/empty/*.txt ./temp/train/

#Clear the folder to be sure no files are left
!rm ./temp/empty/*.txt

!python relate_database_and_images.py \
--image_directory=./temp/test/ \
--wildcards=tif \
--database_directory=./vector_data/ \
--input_databases=v2empty_utm.sqlite\
--input_classes=empty \
--out_directory=./temp/empty/ \
--format=yolo \
--empty_examples=1

# Empty images need empty text files with the same name for YOLO
!for file in ./temp/empty/*.txt; do rm $file; touch $file; done

# Move these files to test folder
!mv ./temp/empty/*.txt ./temp/test/

#Clear the folder to be sure no files are left
!rm ./temp/empty/*.txt

['v2empty_utm.sqlite']
['v2empty_utm']
relate point to mosaic:
Working on:  v2empty_utm.sqlite empty v2empty_utm
Working on  2193 training examples
Comparing with  4146 images
Found  367  matches
Make annotated csv files
/Users/peter/anaconda/anaconda3/envs/geo/lib/python3.6/site-packages/pandas/core/indexing.py:659: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[k] = np.nan
relate_database_and_images.py:204: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp[['x_center_norm', 'y_center_norm', 'widt

In [18]:
#Delete all images that have no label-file
!python yolo_delete_images_with_no_label.py ./temp/train .tif
!python yolo_delete_images_with_no_label.py ./temp/test .tif

100%|████████████████████████████████████| 4146/4146 [00:00<00:00, 11912.62it/s]
Deleted  2511  files.
100%|██████████████████████████████████████| 460/460 [00:00<00:00, 10052.47it/s]
Deleted  284  files.


In [38]:
#Convert to grayscale if needed (do not do that normally, all SSS mosaics were exported as grayscale)
#!python convert_to_greyscale.py ./temp/train ./temp/train tif -o
#!python convert_to_greyscale.py ./temp/test ./temp/test tif -o

Using  15  cores. 
100%|█████████████████████████████████████| 1319/1319 [00:00<00:00, 1770.27it/s]
Following files had error
[]
Using  15  cores. 
100%|████████████████████████████████████████| 134/134 [00:00<00:00, 252.14it/s]
Following files had error
[]


In [19]:
#convert to png and remove tif
!python convert_to_png.py ./temp/train
!python convert_to_png.py ./temp/test

!rm ./temp/train/*.tif
!rm ./temp/test/*.tif

Using  15  cores. 
TIF warnings are supressed
100%|██████████████████████████████████████| 1635/1635 [00:12<00:00, 129.25it/s]
Using  15  cores. 
TIF warnings are supressed
100%|████████████████████████████████████████| 176/176 [00:01<00:00, 106.36it/s]


In [20]:
# Rotate images and labels to make dataset larger

!python yolo_rotate_img_and_boundaries.py ./temp/train .png
!python yolo_rotate_img_and_boundaries.py ./temp/test .png

Using  15  cores. 
100%|██████████████████████████████████████| 1635/1635 [00:03<00:00, 410.67it/s]
Using  15  cores. 
100%|████████████████████████████████████████| 176/176 [00:00<00:00, 200.18it/s]


In [7]:
# Prepare the training
os.chdir("./temp/current_run")   #only works on first execution of cell

#Make the file lists the the YOLO model requires
!realpath ./../train/*.txt > ./../train/train.txt
!realpath ./../test/*.txt > ./../test/test.txt


#Change .txt with .jpg or .png, depending on what was converted to
#Probably the realpath could also be run directly on the images, forgot what was my rationale for this step
#Note:LINUX MAC SED DIFFERENCE: see https://stackoverflow.com/questions/19456518/error-when-using-sed-with-find-command-on-os-x-invalid-command-code
#If you are on a OS X, this probably has nothing to do with the sed command. On the OSX version of sed, the -i option expects an extension argument so your command is actually parsed as the extension argument and the file path is interpreted as the command code.
#Try adding the -e argument explicitly and giving '' as argument to -i:
!sed -i '' -e 's/txt/png/g' ./../train/train.txt
!sed -i '' -e 's/txt/jpg/g' ./../test/test.txt


#Set-up the classes.txt file. The name of all classes must be in here (preferably in the correct order..)
#There is no need to specifiy the "empty" class

!touch ./../train/classes.txt
!echo "stones" >> ./../train/classes.txt

!touch ./../test/classes.txt
!echo "stones" >> ./../test/classes.txt

#Create the obj.names and obj.classes file

#Make sure the old one does not exist
!rm ./obj.names
!rm ./obj.data

#Make new obj.names
!touch ./obj.names

#Add all class names that are present in the data set on obj.names
!echo "stones" >> ./obj.names

#Make new obj.data
!touch ./obj.data
!echo "classes = 1" >> ./obj.data
!echo "train = ./../train/train.txt" >> ./obj.data
!echo "valid = ./../test/test.txt" >> ./obj.data
!echo "names = ./obj.names" >> ./obj.data
!echo "backup = ./" >> ./obj.data


# Copy the standard configuration file to the current_run folder
# Stop after that and modify the config files to your liking...
!cp ./../../yolov4.cfg .
!cp ./../../pre_trained_models/yolov4.weights .


In [6]:
# calculate optimal anchor sizes for configuration file
# refer to git of darknet for details
# These need to go to the config file. 
# You need to install darknet prior to this step


!/Users/peter/opt/darknet/darknet detector calc_anchors obj.data -num_of_clusters 9 -width 512 -height 512 -show


 GPU isn't used 
 OpenCV version: 4.5.2

 num_of_clusters = 9, width = 512, height = 512 
 read labels from 11549 images 
 loaded 	 image: 2218 	 box: 5306

Wrong label: /Users/peter/IOW Marine Geophysik Dropbox/Code/BoulderDetection/temp/train/1_16_29.txt - j = 2, x = 0.000000, y = 0.796875, width = 0.031250, height = 0.062500 
 loaded 	 image: 2222 	 box: 5322

Wrong label: /Users/peter/IOW Marine Geophysik Dropbox/Code/BoulderDetection/temp/train/1_16_29_rot_270.txt - j = 2, x = 0.203125, y = -0.000000, width = 0.062500, height = 0.031250 
 loaded 	 image: 11539 	 box: 37112 	 box: 9057672 	 box: 12273image: 5756 	 box: 15433aded 	 image: 6842 	 box: 19734943 	 box: 24561 image: 9045 	 box: 28628ox: 32007oaded 	 image: 11167 	 box: 35421
 all loaded. 

 calculating k-means++ ...

 iterations = 17 


counters_per_class = 37112

 avg IoU = 84.36 % 

Saving anchors to the file: anchors.txt 
anchors =  33, 35,  47, 43,  51, 57,  62, 62,  83, 54,  55, 91,  73, 75,  95, 91, 124,128
^C


In [11]:
# Do the actual training
# The model references the pre-trained models that can be downloaded from the github site

!/Users/peter/opt/darknet/darknet \
detector train \
./obj.data \
./yolov4.cfg \
./yolov4.conv.137 \
-map -dont_show


 GPU isn't used 
 OpenCV version: 4.5.2
 Prepare additional network for mAP calculation...
mini_batch = 1, batch = 64, time_steps = 1, train = 0 
   layer   filters  size/strd(dil)      input                output
   0 conv     32       3 x 3/ 1    832 x 832 x   3 ->  832 x 832 x  32 1.196 BF
   1 conv     64       3 x 3/ 2    832 x 832 x  32 ->  416 x 416 x  64 6.380 BF
   2 conv     64       1 x 1/ 1    416 x 416 x  64 ->  416 x 416 x  64 1.418 BF
   3 route  1 		                           ->  416 x 416 x  64 
   4 conv     64       1 x 1/ 1    416 x 416 x  64 ->  416 x 416 x  64 1.418 BF
   5 conv     32       1 x 1/ 1    416 x 416 x  64 ->  416 x 416 x  32 0.709 BF
   6 conv     64       3 x 3/ 1    416 x 416 x  32 ->  416 x 416 x  64 6.380 BF
   7 Shortcut Layer: 4,  wt = 0, wn = 0, outputs: 416 x 416 x  64 0.011 BF
   8 conv     64       1 x 1/ 1    416 x 416 x  64 ->  416 x 416 x  64 1.418 BF
   9 route  8 2 	                           ->  416 x 416 x 128 
  10 conv     64      

 104 conv   1024       1 x 1/ 1     26 x  26 x1024 ->   26 x  26 x1024 1.418 BF
 105 conv    512       1 x 1/ 1     26 x  26 x1024 ->   26 x  26 x 512 0.709 BF
 106 conv   1024       3 x 3/ 1     26 x  26 x 512 ->   26 x  26 x1024 6.380 BF
 107 conv    512       1 x 1/ 1     26 x  26 x1024 ->   26 x  26 x 512 0.709 BF
 108 max                5x 5/ 1     26 x  26 x 512 ->   26 x  26 x 512 0.009 BF
 109 route  107 		                           ->   26 x  26 x 512 
 110 max                9x 9/ 1     26 x  26 x 512 ->   26 x  26 x 512 0.028 BF
 111 route  107 		                           ->   26 x  26 x 512 
 112 max               13x13/ 1     26 x  26 x 512 ->   26 x  26 x 512 0.058 BF
 113 route  112 110 108 107 	                   ->   26 x  26 x2048 
 114 conv    512       1 x 1/ 1     26 x  26 x2048 ->   26 x  26 x 512 1.418 BF
 115 conv   1024       3 x 3/ 1     26 x  26 x 512 ->   26 x  26 x1024 6.380 BF
 116 conv    512       1 x 1/ 1     26 x  26 x1024 ->   26 x  26 x 512 0.709 BF

  43 conv    128       1 x 1/ 1    104 x 104 x 128 ->  104 x 104 x 128 0.354 BF
  44 conv    128       3 x 3/ 1    104 x 104 x 128 ->  104 x 104 x 128 3.190 BF
  45 Shortcut Layer: 42,  wt = 0, wn = 0, outputs: 104 x 104 x 128 0.001 BF
  46 conv    128       1 x 1/ 1    104 x 104 x 128 ->  104 x 104 x 128 0.354 BF
  47 conv    128       3 x 3/ 1    104 x 104 x 128 ->  104 x 104 x 128 3.190 BF
  48 Shortcut Layer: 45,  wt = 0, wn = 0, outputs: 104 x 104 x 128 0.001 BF
  49 conv    128       1 x 1/ 1    104 x 104 x 128 ->  104 x 104 x 128 0.354 BF
  50 conv    128       3 x 3/ 1    104 x 104 x 128 ->  104 x 104 x 128 3.190 BF
  51 Shortcut Layer: 48,  wt = 0, wn = 0, outputs: 104 x 104 x 128 0.001 BF
  52 conv    128       1 x 1/ 1    104 x 104 x 128 ->  104 x 104 x 128 0.354 BF
  53 route  52 25 	                           ->  104 x 104 x 256 
  54 conv    256       1 x 1/ 1    104 x 104 x 256 ->  104 x 104 x 256 1.418 BF
  55 conv    512       3 x 3/ 2    104 x 104 x 256 ->   52 x  52 

 148 conv    512       3 x 3/ 1     52 x  52 x 256 ->   52 x  52 x 512 6.380 BF
 149 conv     18       1 x 1/ 1     52 x  52 x 512 ->   52 x  52 x  18 0.050 BF
 150 yolo
[yolo] params: iou loss: ciou (4), iou_norm: 0.07, obj_norm: 1.00, cls_norm: 1.00, delta_norm: 1.00, scale_x_y: 1.10
nms_kind: greedynms (1), beta = 0.600000 
 151 route  147 		                           ->   52 x  52 x 256 
 152 conv    512       3 x 3/ 2     52 x  52 x 256 ->   26 x  26 x 512 1.595 BF
 153 route  152 116 	                           ->   26 x  26 x1024 
 154 conv    512       1 x 1/ 1     26 x  26 x1024 ->   26 x  26 x 512 0.709 BF
 155 conv   1024       3 x 3/ 1     26 x  26 x 512 ->   26 x  26 x1024 6.380 BF
 156 conv    512       1 x 1/ 1     26 x  26 x1024 ->   26 x  26 x 512 0.709 BF
 157 conv   1024       3 x 3/ 1     26 x  26 x 512 ->   26 x  26 x1024 6.380 BF
 158 conv    512       1 x 1/ 1     26 x  26 x1024 ->   26 x  26 x 512 0.709 BF
 159 conv   1024       3 x 3/ 1     26 x  26 x 512 ->   

In [ ]:
#When Done, manually rename the "temp" folder to a suitable name, to have a complete copy of the model training.